In [1]:
import optuna 
study_mode = "single"
if study_mode=='debias':
    study = optuna.create_study(
                study_name='debias_param_search',
                storage='sqlite:///../debias_param_search.db',
                directions=["minimize","maximize"],
                load_if_exists=True)
if study_mode=="single":
    study = optuna.create_study(
                    study_name='baseline_densenet',
                    storage='sqlite:///../baseline_densenet.db',
                    directions=["minimize"],
                    load_if_exists=True)



/home/ramon/miniconda3/envs/jbi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-11-13 09:46:37,981] Using an existing study with name 'baseline_densenet' instead of creating a new one.


In [2]:
import pandas as pd 
df_rows = list() 
if study_mode =='debias': 
    for e in study.trials:
        if e.state == optuna.trial.TrialState.COMPLETE: 
            task_loss,demo_loss = e.values 
            lmbd = e.params['lambda']
            df_rows.append([lmbd,task_loss,demo_loss,e._trial_id])
    sample = pd.DataFrame(df_rows,columns=['lambda','taskLoss','demoLoss','trial_id']) 
    sample['ratio'] = sample['demoLoss']/sample['taskLoss']
    sample = sample.sort_values(by='ratio',ascending=False)
    sample = sample[sample['taskLoss']<1] 
else: 
    for e in study.trials:
        if e.state == optuna.trial.TrialState.COMPLETE: 
            task_loss = e.values 
            df_rows.append([task_loss[0],e.number ])
    sample = pd.DataFrame(df_rows,columns=['taskLoss','trial_id']) 
    sample = sample.sort_values(by='taskLoss',ascending=True)

In [3]:
study.trials

[FrozenTrial(number=0, state=TrialState.FAIL, values=None, datetime_start=datetime.datetime(2023, 11, 12, 15, 52, 35, 635742), datetime_complete=datetime.datetime(2023, 11, 12, 15, 54, 45, 220796), params={'batch_size': 44, 'optimizer': 'AdamW', 'lr': 0.01930706668039461, 'scheduler': 'ReduceLROnPlateau', 'platau_factor': 0.06999999999999999, 'patience': 4}, user_attrs={}, system_attrs={}, intermediate_values={0: 55.87909698486328}, distributions={'batch_size': IntDistribution(high=64, log=False, low=16, step=4), 'optimizer': CategoricalDistribution(choices=('SGD', 'Adam', 'AdamW')), 'lr': FloatDistribution(high=0.1, log=True, low=1e-05, step=None), 'scheduler': CategoricalDistribution(choices=('ReduceLROnPlateau', 'StepLR')), 'platau_factor': FloatDistribution(high=0.09, log=False, low=0.01, step=0.02), 'patience': IntDistribution(high=10, log=False, low=2, step=2)}, trial_id=1, value=None),
 FrozenTrial(number=1, state=TrialState.FAIL, values=None, datetime_start=datetime.datetime(20

In [4]:
sample

,taskLoss,trial_id
4,1.138479,8
5,1.539353,9
2,5.062725,4
3,6.035272,5
0,23.281195,2
1,155.795258,3


In [30]:
best_trial_idx = sample['trial_id'].iloc[0]

In [33]:
if study_mode=='single': 
    print(study.best_trial.params)
else: 
    print(study.trials[best_trial_idx].params)

{'batch_size': 44, 'lambda': 0.7908505984797102, 'optimizer': 'AdamW', 'lr': 0.00015530268483585034, 'scheduler': 'ReduceLROnPlateau', 'platau_factor': 0.06999999999999999, 'patience': 2}
